<a href="https://colab.research.google.com/github/akoo-45/AppliedDeepLearning/blob/master/resnet14b_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook runs a ResNet14b convolutional neural networks for classifying 5 particle images in a simulated LArTPC detector available from the [public dataset](http://deeplearnphysics.org/DataChallenge/). We use Keras to train the network and larcv_threadio to fetch data from larcv files.




```
ssh hopper
cd /data/ashley.koo/larcv-tutorial
# Download necessary libraries (see appendix in [writeup]
(https://docs.google.com/document/d/1jElkhcZG15OG6Azza3dgda2aagX1vHUS5YlMw8LcOBc/edit)) 
python akoo_resnet14b_tf.py
```







# Imports

In [0]:
from larcv import larcv
from larcv.dataloader2 import larcv_threadio
import matplotlib.pyplot as plt
import numpy as np
import os,sys,time
import six
import tensorflow as tf
import tensorflow.contrib.keras as keras 

from keras.models import Model, load_model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
	   Dropout 
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)
from keras.layers.merge import Add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

# Configurations

In [0]:
# Tensorflow/gpu start-up configuration
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='2'

# Making directory paths
TUTORIAL_DIR     = '.'
TRAIN_IO_CONFIG  = os.path.join(TUTORIAL_DIR, 'tf/io_train.cfg') # configuration file for train_io stored in './tf/io_train.cfg'
TEST_IO_CONFIG   = os.path.join(TUTORIAL_DIR, 'tf/io_test.cfg' ) # configuration file for test_io
TRAIN_BATCH_SIZE = 10 # Unused variable
TEST_BATCH_SIZE  = 100 # Unused variable 
LOGDIR           = 'resnet_log' 
ITERATIONS       = 10 # Unused variable

# Check that the log directory is empty 
train_logdir = os.path.join(LOGDIR,'train')
test_logdir  = os.path.join(LOGDIR,'test')

if not os.path.isdir(train_logdir): os.makedirs(train_logdir)
if not os.path.isdir(test_logdir):  os.makedirs(test_logdir)
if len(os.listdir(train_logdir)) or len(os.listdir(test_logdir)):
  sys.stderr.write('Error: train or test log dir not empty...\n')
  raise OSError

# Configuring Data Reader 
We prepare two data reader instances: one for training and another for testing the network. 

In [0]:
#
# Step 0: Configure the IO readers
#

# Training dataset 
train_io = larcv_threadio()  
train_io_cfg = {'filler_name' : 'TrainIO',
                'verbosity'   : 10,
                'filler_cfg'  : TRAIN_IO_CONFIG}
train_io.configure(train_io_cfg)   
train_io.start_manager(TRAIN_BATCH_SIZE) # arg: # of images to store
time.sleep(2)
train_io.next() # transforms data into numpy array

# Test dataset 
test_io = larcv_threadio()   # create io interface
test_io_cfg = {'filler_name' : 'TestIO',
               'verbosity'   : 10,
               'filler_cfg'  : TEST_IO_CONFIG}
test_io.configure(test_io_cfg)   # configure
test_io.start_manager(TEST_BATCH_SIZE) # start read thread
time.sleep(2) 
test_io.next()

#Defining a network
We use 4 resnet blocks for ResNet14b. 

Referenced Kaiming He's repository, found here: 
https://github.com/KaimingHe/deep-residual-networks



In [0]:
# 
# Step 1: Define ResNet14b Network
#
def _bn_relu(input):
    """Helper to build a BN (batch normalization) -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)

def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1)) # Overrides strides?
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same") # Padding, SAME????????? TODO
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f

# TODO: reduce code later by duplicating with _conv_bn_relu
def _conv_bn(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1)) # Overrides strides?
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same") # Padding, SAME?????????
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return BatchNormalization(axis=CHANNEL_AXIS)(conv)

    return f


def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f

def _shortcut(input, residual):
    """Adds a shortcut between input (pool) and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    print("Entering shortcut")
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    print (input_shape)
    print (residual_shape) 
    print ("Input channels ", input_shape[CHANNEL_AXIS])
    print ("Residual channels ", residual_shape[CHANNEL_AXIS])
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        print("The number of filters ", residual_shape[CHANNEL_AXIS]) # TODO
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])

def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input): 
	      print( "Entering residual block" ) 
        for i in range(repetitions):
            init_strides = (1, 1) 
            if i == 0 and not is_first_layer: # NOTE: this was confusing to interpret
                init_strides = (2, 2) # Why strides = (2,2) for the rest of the layers?
            input = block_function(filters=filters, init_strides=init_strides,  
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input

    return f

def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4
    """
    def f(input):
        # if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool # you still did it... (possible optimization) TODO **
        #     conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
        #                       strides=init_strides,
        #                       padding="same",
        #                       kernel_initializer="he_normal",
        #                       kernel_regularizer=l2(1e-4))(input)
        # else:
            # revert later ** changed all 3  _bn_relu_conv --> _conv_bn_relu (possible optimization) TODO
            # see if padding needs to change TODO padding = same for ALL 
        print("Entering bottleneck stage")
	      print("Filters", filters, "strides", init_strides,"before conv_bn_relu")
        conv_1_1 = _conv_bn_relu(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input) # Pass on strides!?
        conv_3_3 = _conv_bn_relu(filters=filters, kernel_size=(3, 3))(conv_1_1)
        # filters = filters * 4 is true if you check the ethereon model for Resnet 50 
        # might have to edit ** last layer doesn't have relu...  ** TODO
        # residual = _conv_bn_relu(filters=filters * 4, kernel_size=(1, 1))(conv_3_3) # THIS HAS DIFFERENT PADDING THOUGH ?? Does padding = same do the trick??
        residual = _conv_bn(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        print("Filter used for the residual layer ",  filters * 4)
        return _shortcut(input, residual)

    return f

def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS

	  # If K backend engine is tf
	  CHANNEL_AXIS = 1
	  ROW_AXIS = 2
	  COL_AXIS = 3

def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary *
        # if K.image_data_format() == 'tf':
        #     input_shape = (input_shape[1], input_shape[2], input_shape[0]) # rows, cols, samples? *?

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape) # creation of keras tensor  
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input) # padding = 3? *? TODO
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions): # handy-dandy!
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block) # block is input for bottleneck layer 
            filters *= 2
	      block = _bn_relu(block)

        # Last activation
        # I think this is just _relu... for the last block  but will leave it TODO
        block = _bn_relu(block) 

        # Classifier block
        # changed from strides=(1,1) to (18,18) ** TODO don't know why kernel size doesn't show up in Kazu's... 
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(18, 18))(block) # where is the kernel?? 7?? TODO
	      flatten1 = Flatten()(pool2)  
	      dropout = Dropout(rate=0.2)(flatten1) 
	      dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax")(dropout) 

        model = Model(inputs=input, outputs=dense)
	      print("returning model")
        return model

    
    # Custom method for ResNet14b 
    @staticmethod
    def build_resnet_14b(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [1, 1, 1, 1])

    # @staticmethod
    # def build_resnet_18(input_shape, num_outputs):
    #     return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    # **
    # @staticmethod
    # def build_resnet_34(input_shape, num_outputs):
    #     return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    #     @staticmethod
    #     def build_resnet_50(input_shape, num_outputs):
    #         return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    # @staticmethod
    # def build_resnet_101(input_shape, num_outputs):
    #     return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    # @staticmethod
    # def build_resnet_152(input_shape, num_outputs):


# Build the Network

In [0]:
#
# Step 2: Build network + define loss & solver
#
# retrieve dimensions of data for network construction
img_rows, img_cols = 256, 256
img_channels = 1
num_classes = 5
batch_size = 50
epochs = 5


model = ResnetBuilder.build_resnet_14b(input_shape=(img_rows, img_cols, img_channels), num_outputs=num_classes)

#from keras.models import Sequential 
#model = Sequential()
# Trying simple model first, to see if it works for .evaluate() on test datsemodel.add(Conv2D(64, (3, 3), input_shape=(1, 256, 256, padding='same',))
#model.add(Conv2D(64, (1, 1), input_shape=(256, 256, 1)))
#model.add(Dropout(0.25))
#model.add(Flatten())
#model.add(Dense(units=5, activation='softmax'))
#model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print ("Configured model for training")

# Create a callback Tensorboard object 
keras.callbacks.TensorBoard(log_dir='./Graph_Batch50_Epoch5', histogram_freq=0,  
          write_graph=True, write_images=True)

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph_Batch50_Epoch5', histogram_freq=0, write_graph=True, write_images=True)

# Train!

In [0]:
#
# Step 4: Train the model 
#
# Fetch python array data from original data  

# Train_IO
train_data  = train_io.fetch_data('train_image').data()
train_label = train_io.fetch_data('train_label').data()
dim_data = train_io.fetch_data('train_image').dim()

train_data_reshaped = train_data.reshape(dim_data)
test_data  = test_io.fetch_data('test_image').data()
test_label = test_io.fetch_data('test_label').data()
test_data_reshaped = test_data.reshape(dim_data)


history = model.fit(train_data_reshaped, train_label, 
	batch_size=batch_size, 
	epochs=epochs, 
	verbose=1, 
	#validation_split=0.4
    validation_data=(test_data_reshaped, test_label),
    callbacks=[tbCallBack]	# Gave your callback object to your function 
	)

# Switching between Keras' training and inference modes

The batch normalization layer is necessary after each convolutional layer. However, when run in Keras. 



In [0]:
print("BEFORE SAVING MODEL")
K.set_learning_phase(0)
scores = model.evaluate(test_data_reshaped, test_label, verbose=0)
print('Test dataset loss:', scores[0])
print('Test dataset accuracy:', scores[1])

model.save('tmp.v1')

# In every test we will clear the session and reload the model to force Learning_Phase values to change
print('DYNAMIC LEARNING PHASE')
K.clear_session()
model = load_model('tmp.v1')
scores = model.evaluate(test_data_reshaped, test_label, verbose=1)
print('Test dataset loss:', scores[0])
print('Test dataset accuracy:', scores[1])

print('STATIC LEARNING PHASE = 0 TEST MODE')
K.clear_session()
model = load_model('tmp.v1')
K.set_learning_phase(0)
scores = model.evaluate(test_data_reshaped, test_label, verbose=1)
print('Test dataset loss:', scores[0])
print('Test dataset accuracy:', scores[1])

print('STATIC LEARNING PHASE = 1 TRAIN MODE')
K.clear_session()
model = load_model('tmp.v1')
K.set_learning_phase(1)

scores = model.evaluate(test_data_reshaped, test_label, verbose=1)
print('Test dataset loss:', scores[0])
print('Test dataset accuracy:', scores[1])

# Plot Accuracies & Loss! 

In [0]:
# Plot Accuracy & Loss
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, epochs+1)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
# scores = model.evaluate(test_data_reshaped, test_label, verbose=0)
# scores = model.evaluate(train_data_reshaped, train_label, verbose=0)

#print('Test dataset loss:', scores[0])
#print('Test dataset accuracy:', scores[1])

#inform log directory
print()
print("You are done") 
train_io.reset()
test_io.reset()